In [1]:
install_fuse = True  # @param {type:"boolean"}
use_gpu = True  # @param {type:"boolean"}

if install_fuse:
    !git clone https://github.com/MichalWeinbaum/fuse-med-ml.git  
    # !git clone https://github.com/BiomedSciAI/fuse-med-ml.git
    %cd fuse-med-ml
    %pip install -e .[all,examples]

fatal: destination path 'fuse-med-ml' already exists and is not an empty directory.
/content/fuse-med-ml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fuse-med-ml
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fuse-med-ml (pyproject.toml) ... done
  Created wheel for fuse-med-ml: filename=fuse_med_ml-0.3.0-0.editable-py3-none-any.whl size=13432 sha256=fa71b0a2716c3fc2749006191f3119fe5f5299a1f6ab26b52d347da0641c945f
  Stored in directory: /tmp/pip-ephem-wheel-cache-elnksrnt/wheels/b1/15/3b/51083b9282297d7924724e28615f2ac7d7b48fef3cf512f18e
Successfully built fuse-med-ml
  Attempting uninstall: fuse-med-ml
    Found existing installation: fuse-med-ml 0.3.0
    Uninstalling fuse-med-ml-0.3.0:
      Successfully uninstalled fuse-m

In [ ]:
!pip install torch

In [ ]:
!pip install pytorch_lightning

In [ ]:
!pip install hdf5plugin

In [ ]:
!pip install --upgrade torchvision

In [ ]:
!pip install deepdiff

In [ ]:
!pip install clearml

In [ ]:
!pip install wget

In [ ]:
!pip install nibabel

In [ ]:
!pip install medpy

In [ ]:
!pip install pydicom

In [ ]:
!pip install vit_pytorch

In [ ]:
!pip install --user torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join(r'C:\Users\USER\Documents\fuse-med-ml-master-1'))
if module_path not in sys.path:
    sys.path.append(module_path)
print (module_path)

/content/fuse-med-ml/C:\Users\USER\Documents\fuse-med-ml-master-1


In [4]:
"""
(C) Copyright 2021 IBM Corp.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

Created on June 30, 2021

"""

import os
import copy
import logging
from typing import OrderedDict, Sequence, Tuple

import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader

from fuse.dl.models import ModelMultiHead
from fuse.dl.models.backbones.backbone_resnet import BackboneResnet
from fuse.dl.models.heads.head_global_pooling_classifier import HeadGlobalPoolingClassifier
from fuse.dl.models.backbones.backbone_inception_resnet_v2 import BackboneInceptionResnetV2

from fuse.eval.metrics.classification.metrics_thresholding_common import MetricApplyThresholds
from fuse.eval.metrics.classification.metrics_classification_common import MetricAccuracy, MetricAUCROC, MetricROCCurve

from fuse.utils.utils_debug import FuseDebug
from fuse.utils.utils_logger import fuse_logger_start
from fuse.utils.file_io.file_io import create_dir, save_dataframe, load_pickle
import fuse.utils.gpu as GPU

from fuse.eval.evaluator import EvaluatorDefault
from fuse.dl.losses.loss_default import LossDefault
from fuse.data.utils.collates import CollateDefault

import pytorch_lightning as pl
from fuse.dl.lightning.pl_module import LightningModuleDefault
from fuse.dl.lightning.pl_funcs import convert_predictions_to_dataframe
from pytorch_lightning.utilities.rank_zero import rank_zero_only

from fuseimg.datasets.isic import ISIC, ISICDataModule

from fuse_examples.imaging.classification.isic.golden_members import FULL_GOLDEN_MEMBERS, TEN_GOLDEN_MEMBERS

import torch.nn as nn
from fuse.dl.models.model_wrapper import ModelWrapSeqToDict
from fuse.dl.models.backbones.backbone_vit import ViT


In [5]:
###########################################################################################################
# Fuse
###########################################################################################################
##########################################
# Debug modes
##########################################
###Michal change to ndebug 
mode = "default"  # Options: '', 'debug'. See details in FuseDebug
debug = FuseDebug(mode)

##########################################
# GPUs and Workers
##########################################
###Michal change GPUs from 1 to 0 and back to 1 (for cpu)
NUM_GPUS = 1 # supports multiple gpu training with DDP strategy
NUM_WORKERS = 0

##########################################
# Modality
##########################################
multimodality = False  # Set: 'False' to use only imaging, 'True' to use imaging & meta-data

##########################################
# Model Type
##########################################
model_type = "CNN"  # Set: 'Transformer' to use ViT/MMViT, 'CNN' to use InceptionResNet

##########################################
# Output Paths
##########################################


ROOT = "./_examples/isic/"
DATA = os.environ["ISIC19_DATA_PATH"] if "ISIC19_DATA_PATH" in os.environ else os.path.join(ROOT, "data_dir")
modality = "multimodality" if multimodality else "imaging"
model_dir = os.path.join(ROOT, f"model_dir_{modality}")
PATHS = {
    "model_dir": model_dir,
    "inference_dir": os.path.join(model_dir, "infer_dir"),
    "eval_dir": os.path.join(model_dir, "eval_dir"),
    "data_dir": DATA,
    "cache_dir": os.path.join(ROOT, "cache_dir"),
    "data_split_filename": os.path.join(model_dir, "isic_split.pkl"),
}


##########################################
# Train Common Params
##########################################
TRAIN_COMMON_PARAMS = {}
# ============
# Data
# ============
TRAIN_COMMON_PARAMS["data.batch_size"] = 32  # effective batch size = batch_size * num_gpus
TRAIN_COMMON_PARAMS["data.num_workers"] = NUM_WORKERS
TRAIN_COMMON_PARAMS["data.num_folds"] = 5
TRAIN_COMMON_PARAMS["data.train_folds"] = [0, 1, 2]
TRAIN_COMMON_PARAMS["data.validation_folds"] = [3]
TRAIN_COMMON_PARAMS["data.infer_folds"] = [4]
TRAIN_COMMON_PARAMS["data.samples_ids"] = {"all": None, "golden": FULL_GOLDEN_MEMBERS}["golden"]


# ===============
# PL Trainer
# ===============
TRAIN_COMMON_PARAMS["trainer.num_epochs"] = 20
TRAIN_COMMON_PARAMS["trainer.num_devices"] = NUM_GPUS
###Michal
TRAIN_COMMON_PARAMS["trainer.accelerator"] = "gpu"
TRAIN_COMMON_PARAMS["trainer.strategy"] = "ddp" if NUM_GPUS > 1 else None

# ===============
# Optimizer
# ===============
TRAIN_COMMON_PARAMS["opt.lr"] = 1e-5
TRAIN_COMMON_PARAMS["opt.weight_decay"] = 1e-3

# ===============
# Model
# ===============
if model_type == "CNN":
    TRAIN_COMMON_PARAMS["model"] = dict(
        dropout_rate=0.5,
        layers_description=(256,),
        tabular_data_inputs=[("data.input.clinical.all", 19)] if multimodality else None,
        tabular_layers_description=(128,) if multimodality else tuple(),
    )
elif model_type == "Transformer":
    token_dim = 768
    TRAIN_COMMON_PARAMS["model"] = dict(
        token_dim=token_dim,
        projection_kwargs=dict(image_shape=[300, 300], patch_shape=[30, 30], channels=3),
        transformer_kwargs=dict(depth=12, heads=12, mlp_dim=token_dim * 4, dim_head=64, dropout=0.0, emb_dropout=0.0),
    )

In [6]:
print (TRAIN_COMMON_PARAMS)

{'data.batch_size': 32, 'data.num_workers': 0, 'data.num_folds': 5, 'data.train_folds': [0, 1, 2], 'data.validation_folds': [3], 'data.infer_folds': [4], 'data.samples_ids': ['ISIC_0072637', 'ISIC_0072638', 'ISIC_0072639', 'ISIC_0072640', 'ISIC_0072641', 'ISIC_0072642', 'ISIC_0072646', 'ISIC_0072647', 'ISIC_0072648', 'ISIC_0072649', 'ISIC_0072650', 'ISIC_0072651', 'ISIC_0072652', 'ISIC_0072653', 'ISIC_0072654', 'ISIC_0072656', 'ISIC_0072657', 'ISIC_0072658', 'ISIC_0072659', 'ISIC_0072660', 'ISIC_0072661', 'ISIC_0072662', 'ISIC_0072663', 'ISIC_0072665', 'ISIC_0072666', 'ISIC_0072668', 'ISIC_0072670', 'ISIC_0072673', 'ISIC_0072675', 'ISIC_0072676', 'ISIC_0072679', 'ISIC_0072683', 'ISIC_0072687', 'ISIC_0072688', 'ISIC_0072690', 'ISIC_0072691', 'ISIC_0072693', 'ISIC_0072694', 'ISIC_0072697', 'ISIC_0072698', 'ISIC_0072699', 'ISIC_0072701', 'ISIC_0072703', 'ISIC_0072708', 'ISIC_0072709', 'ISIC_0072710', 'ISIC_0072713', 'ISIC_0072714', 'ISIC_0072715', 'ISIC_0072716', 'ISIC_0072717', 'ISIC_007

In [7]:
def perform_softmax(logits: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    cls_preds = F.softmax(logits, dim=1)
    return logits, cls_preds

In [8]:
class MMViT(ViT):
    def __init__(self, token_dim: int, projection_kwargs: dict, transformer_kwargs: dict, multimodality: bool):
        super().__init__(token_dim, projection_kwargs, transformer_kwargs)
        self.multimodality = multimodality
        num_tokens = self.projection_layer.num_tokens
        self.token_dim = token_dim
        self._head = nn.Linear(token_dim, 8)
        if self.multimodality:
            # change pos embedding to accept additional token for multimodal
            self.transformer.pos_embedding = nn.Parameter(torch.randn(1, num_tokens + 2, token_dim))

    # This forward can be Multimodal or just Imaging
    def forward(self, img_x: torch.Tensor, clinical_x: torch.Tensor = None) -> torch.Tensor:
        img_x = self.projection_layer(img_x)
        if self.multimodality:
            clinical_x = clinical_x.unsqueeze(1)
            clinical_x_zeros = torch.zeros((img_x.shape[0], 1, self.token_dim))
            clinical_x_zeros[:, :, :19] = clinical_x
            clinical_x = clinical_x_zeros.cuda()
            x = torch.cat((img_x, clinical_x), 1)
        else:
            x = img_x
        x = self.transformer(x)
        x = self._head(x[:, 0])
        return x

In [9]:
def create_transformer_model(
    token_dim: int,
    projection_kwargs: dict,
    transformer_kwargs: dict,
) -> ModelWrapSeqToDict:
    torch_model = MMViT(
        token_dim=token_dim,
        projection_kwargs=projection_kwargs,
        transformer_kwargs=transformer_kwargs,
        multimodality=multimodality,
    )
    model = ModelWrapSeqToDict(
        model=torch_model,
        model_inputs=["data.input.img", "data.input.clinical.all"] if multimodality else ["data.input.img"],
        post_forward_processing_function=perform_softmax,
        model_outputs=["model.logits.head_0", "model.output.head_0"],
    )
    return model

In [10]:
def create_cnn_model(
    dropout_rate: float,
    layers_description: Sequence[int],
    tabular_data_inputs: Sequence[Tuple[str, int]],
    tabular_layers_description: Sequence[int],
) -> torch.nn.Module:
    """
    creates the model
    """
    print ("before ModelMultiHead")
    model = ModelMultiHead(
        conv_inputs=(("data.input.img", 3),),
        backbone={
            "Resnet18": BackboneResnet(pretrained=True, in_channels=3, name="resnet18"),
            "InceptionResnetV2": BackboneInceptionResnetV2(input_channels_num=3, logical_units_num=43),
        }["InceptionResnetV2"],
        heads=[
            HeadGlobalPoolingClassifier(
                head_name="head_0",
                dropout_rate=dropout_rate,
                conv_inputs=[("model.backbone_features", 1536)],
                tabular_data_inputs=tabular_data_inputs,
                layers_description=layers_description,
                tabular_layers_description=tabular_layers_description,
                num_classes=8,
                pooling="avg",
            ),
        ],
    )
    return model

In [11]:
print (PATHS)

{'model_dir': './_examples/isic/model_dir_imaging', 'inference_dir': './_examples/isic/model_dir_imaging/infer_dir', 'eval_dir': './_examples/isic/model_dir_imaging/eval_dir', 'data_dir': './_examples/isic/data_dir', 'cache_dir': './_examples/isic/cache_dir', 'data_split_filename': './_examples/isic/model_dir_imaging/isic_split.pkl'}


In [12]:
def create_datamodule(paths: dict, train_common_params: dict) -> pl.LightningDataModule:
    """
    In order to support the DDP strategy one need to create a Lightning Data Module.
    """
    print ("before ISICDataModule")
    datamodule = ISICDataModule(
        data_dir=paths["data_dir"],
        cache_dir=paths["cache_dir"],
        num_workers=train_common_params["data.num_workers"],
        batch_size=train_common_params["data.batch_size"],
        train_folds=train_common_params["data.train_folds"],
        validation_folds=train_common_params["data.validation_folds"],
        infer_folds=train_common_params["data.infer_folds"],
        split_filename=paths["data_split_filename"],
        sample_ids=train_common_params["data.samples_ids"],
        reset_cache=False,
        reset_split=False,
        use_batch_sampler=True if NUM_GPUS <= 1 else False,
    )

    return datamodule

In [13]:
#################################
# Train Template
#################################
def run_train(paths: dict, train_common_params: dict) -> None:
    # ==============================================================================
    # Logger
    # ==============================================================================
    fuse_logger_start(output_path=paths["model_dir"], console_verbose_level=logging.INFO)
    lgr = logging.getLogger("Fuse")
    lgr.info("Fuse Train", {"attrs": ["bold", "underline"]})

    lgr.info(f'model_dir={paths["model_dir"]}', {"color": "magenta"})
    lgr.info(f'data_dir={paths["data_dir"]}', {"color": "magenta"})
    lgr.info(f'cache_dir={paths["cache_dir"]}', {"color": "magenta"})

    # ==============================================================================
    # Data
    # ==============================================================================
    lgr.info("Datamodule:", {"attrs": "bold"})

    datamodule = create_datamodule(paths, train_common_params)

    lgr.info("Datamodule: Done", {"attrs": "bold"})

    # ==============================================================================
    # Model
    # ==============================================================================
    lgr.info("Model:", {"attrs": "bold"})

    if model_type == "Transformer":
        model = create_transformer_model(**train_common_params["model"])
    elif model_type == "CNN":
        model = create_cnn_model(**train_common_params["model"])

    lgr.info("Model: Done", {"attrs": "bold"})

    # ====================================================================================
    #  Loss
    # ====================================================================================
    losses = {
        "cls_loss": LossDefault(pred="model.logits.head_0", target="data.label", callable=F.cross_entropy, weight=1.0),
    }

    # ====================================================================================
    # Metrics
    # ====================================================================================
    class_names = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC"]
    train_metrics = OrderedDict(
        [
            ("op", MetricApplyThresholds(pred="model.output.head_0")),  # will apply argmax
            ("auc", MetricAUCROC(pred="model.output.head_0", target="data.label", class_names=class_names)),
            ("accuracy", MetricAccuracy(pred="results:metrics.op.cls_pred", target="data.label")),
        ]
    )

    validation_metrics = copy.deepcopy(train_metrics)  # use the same metrics in validation as well

    best_epoch_source = dict(monitor="validation.metrics.auc.macro_avg", mode="max")

    # create optimizer
    optimizer = optim.Adam(
        model.parameters(), lr=train_common_params["opt.lr"], weight_decay=train_common_params["opt.weight_decay"]
    )

    # create learning scheduler
    lr_scheduler = {
        "ReduceLROnPlateau": optim.lr_scheduler.ReduceLROnPlateau(optimizer),
        "CosineAnnealing": optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1),
    }["ReduceLROnPlateau"]
    lr_sch_config = dict(scheduler=lr_scheduler, monitor="validation.losses.total_loss")

    # optimizer and lr sch - see pl.LightningModule.configure_optimizers return value for all options
    optimizers_and_lr_schs = dict(optimizer=optimizer, lr_scheduler=lr_sch_config)

    # =====================================================================================
    #  Train
    # =====================================================================================
    lgr.info("Train:", {"attrs": "bold"})

    # create instance of PL module - FuseMedML generic version
    print ("before LightningModuleDefault")
    pl_module = LightningModuleDefault(
        model_dir=paths["model_dir"],
        model=model,
        losses=losses,
        train_metrics=train_metrics,
        validation_metrics=validation_metrics,
        best_epoch_source=best_epoch_source,
        optimizers_and_lr_schs=optimizers_and_lr_schs,
    )

    # create lightning trainer.
    print ("before pl.Trainer")
    pl_trainer = pl.Trainer(
        default_root_dir=paths["model_dir"],
        max_epochs=train_common_params["trainer.num_epochs"],
        accelerator=train_common_params["trainer.accelerator"],
        devices=train_common_params["trainer.num_devices"],
        strategy=train_common_params["trainer.strategy"],
    )

    # train
    print ("before fit")
    pl_trainer.fit(pl_module, datamodule=datamodule)

    lgr.info("Train: Done", {"attrs": "bold"})


In [14]:
######################################
# Inference Common Params
######################################
INFER_COMMON_PARAMS = {}
INFER_COMMON_PARAMS["infer_filename"] = "infer_file.gz"
INFER_COMMON_PARAMS["checkpoint"] = "best_epoch.ckpt"
INFER_COMMON_PARAMS["data.num_workers"] = NUM_WORKERS
INFER_COMMON_PARAMS["data.infer_folds"] = [4]  # infer validation set
INFER_COMMON_PARAMS["data.batch_size"] = 4

INFER_COMMON_PARAMS["model"] = TRAIN_COMMON_PARAMS["model"]
INFER_COMMON_PARAMS["trainer.num_devices"] = 1  # No need for multi-gpu in inference
INFER_COMMON_PARAMS["trainer.accelerator"] = TRAIN_COMMON_PARAMS["trainer.accelerator"]

######################################
# Inference Template
######################################


@rank_zero_only
def run_infer(paths: dict, infer_common_params: dict) -> None:
    create_dir(paths["inference_dir"])
    infer_file = os.path.join(paths["inference_dir"], infer_common_params["infer_filename"])
    checkpoint_file = os.path.join(paths["model_dir"], infer_common_params["checkpoint"])

    ## Logger
    fuse_logger_start(output_path=paths["inference_dir"], console_verbose_level=logging.INFO)
    lgr = logging.getLogger("Fuse")
    lgr.info("Fuse Inference", {"attrs": ["bold", "underline"]})
    lgr.info(f"infer_filename={infer_file}", {"color": "magenta"})

    ## Data
    folds = load_pickle(paths["data_split_filename"])  # assume exists and created in train func

    infer_sample_ids = []
    for fold in infer_common_params["data.infer_folds"]:
        infer_sample_ids += folds[fold]

    # Create dataset
    infer_dataset = ISIC.dataset(paths["data_dir"], paths["cache_dir"], samples_ids=infer_sample_ids, train=False)

    # dataloader
    infer_dataloader = DataLoader(
        dataset=infer_dataset,
        collate_fn=CollateDefault(),
        batch_size=infer_common_params["data.batch_size"],
        num_workers=infer_common_params["data.num_workers"],
    )

    # load python lightning module
    if model_type == "Transformer":
        model = create_transformer_model(**infer_common_params["model"])
    elif model_type == "CNN":
        model = create_cnn_model(**infer_common_params["model"])

    pl_module = LightningModuleDefault.load_from_checkpoint(
        checkpoint_file, model_dir=paths["model_dir"], model=model, map_location="cpu", strict=True
    )
    # set the prediction keys to extract (the ones used be the evaluation function).
    pl_module.set_predictions_keys(["model.output.head_0", "data.label"])  # which keys to extract and dump into file

    # create a trainer instance
    pl_trainer = pl.Trainer(
        default_root_dir=paths["model_dir"],
        accelerator=infer_common_params["trainer.accelerator"],
        devices=infer_common_params["trainer.num_devices"],
        auto_select_gpus=True,
        max_epochs=0,
    )
    predictions = pl_trainer.predict(pl_module, infer_dataloader, return_predictions=True)

    # convert list of batch outputs into a dataframe
    infer_df = convert_predictions_to_dataframe(predictions)
    save_dataframe(infer_df, infer_file)


In [15]:
######################################
# Eval Common Params
######################################
EVAL_COMMON_PARAMS = {}
EVAL_COMMON_PARAMS["infer_filename"] = INFER_COMMON_PARAMS["infer_filename"]


######################################
# Eval Template
######################################
@rank_zero_only
def run_eval(paths: dict, eval_common_params: dict) -> None:
    infer_file = os.path.join(paths["inference_dir"], eval_common_params["infer_filename"])

    fuse_logger_start(output_path=None, console_verbose_level=logging.INFO)
    lgr = logging.getLogger("Fuse")
    lgr.info("Fuse Eval", {"attrs": ["bold", "underline"]})

    # metrics
    metrics = OrderedDict(
        [
            ("op", MetricApplyThresholds(pred="model.output.head_0")),  # will apply argmax
            ("auc", MetricAUCROC(pred="model.output.head_0", target="data.label")),
            ("accuracy", MetricAccuracy(pred="results:metrics.op.cls_pred", target="data.label")),
            (
                "roc",
                MetricROCCurve(
                    pred="model.output.head_0",
                    target="data.label",
                    output_filename=os.path.join(paths["inference_dir"], "roc_curve.png"),
                ),
            ),
        ]
    )

    # create evaluator
    evaluator = EvaluatorDefault()

    # run
    results = evaluator.eval(ids=None, data=infer_file, metrics=metrics, output_dir=paths["eval_dir"])

    return results


In [ ]:
######################################
# Run
######################################
if __name__ == "__main__":
    ## allocate gpus
    # uncomment if you want to use specific gpus instead of automatically looking for free ones
    force_gpus = None  # [0]
    GPU.choose_and_enable_multiple_gpus(NUM_GPUS, force_gpus=force_gpus)

    ISIC.download(data_path=PATHS["data_dir"])

os.environ['CUDA_VISIBLE_DEVICES'] = 0

Extract ISIC-2019 training input ... (this may take a few minutes)
Extract to ./_examples/isic/data_dir/ISIC2019/ISIC_2019_Training_Input


In [ ]:
    RUNNING_MODES = ["train", "infer", "eval"]  # Options: 'train', 'infer', 'eval'

    # train
    if "train" in RUNNING_MODES:
        run_train(paths=PATHS, train_common_params=TRAIN_COMMON_PARAMS)

Fuse Train
model_dir=./_examples/isic/model_dir_imaging
data_dir=./_examples/isic/data_dir
cache_dir=./_examples/isic/cache_dir
Datamodule:
before ISICDataModule
ISIC dataset 
ISIC download 
ISIC static_pipeline 
in load dataframe file:  ./_examples/isic/data_dir\ISIC2019/ISIC_2019_Training_GroundTruth.csv
in load dataframe file:  ./_examples/isic/data_dir\ISIC2019/ISIC_2019_Training_Metadata.csv
before printing static
0@init_@call___call__@@<module 'fuseimg.datasets.isic' from 'C:\\Users\\USER\\Documents\\fuse-med-ml-master-1\\fuseimg\\datasets\\isic.py'>@    def __call__(self, sample_dict: NDict) -> NDict:
        """
        decodes sample id into image file name
        """
        sid = get_sample_id(sample_dict)
        print ("in ISIC OpISICSampleIDDecode _call_ sid= ")
        img_filename_key = "data.input.img_path"
        sample_dict[img_filename_key] = sid + ".jpg"

        return sample_dict
@{}@1@init_@call___call__@format@infer@key_metadata_out@None@<module 'fuseimg.data

C:\Users\USER\Documents\fuse-med-ml-master-1\fuse\data\datasets\caching\samples_cacher.py:110: UserWarning: Multi processing is not active in SamplesCacher. Seting "workers" to the number of your cores usually results in a significant speedup. Debugging, however, is easier with "workers=0".
  warn(
caching:   0%|          | 0/10 [00:00<?, ?it/s]

in ISIC OpISICSampleIDDecode _call_ sid= 
OpLoadImage, img file:  ./_examples/isic/data_dir\ISIC2019/ISIC_2019_Training_Input\ISIC_0072637.jpg jpg


caching:  10%|█         | 1/10 [00:03<00:32,  3.63s/it]

in save safe hdf5 file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@bf35a2a43e5797c123fd45586d96b83a.hdf5f9739b6fd66a90f7ac546dacd975af6c.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@bf35a2a43e5797c123fd45586d96b83a.pkl.gzfb6dee5f764bb0ecd5e9ed869f4402c0.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_info_for_orig_sample@bf35a2a43e5797c123fd45586d96b83a.pkla5c6d390a58181d22362c1169c84a19e.scramlbed
in ISIC OpISICSampleIDDecode _call_ sid= 
OpLoadImage, img file:  ./_examples/isic/data_dir\ISIC2019/ISIC_2019_Training_Input\ISIC_0072638.jpg jpg


...

in save safe hdf5 file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@726d84ed4e7280039be9c0e8ee30e9b0.hdf52648a4f93720d3dd7e6f905f6a2faf4f.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@726d84ed4e7280039be9c0e8ee30e9b0.pkl.gzd30d071c8a3e95bfd5aced929ca1d2c8.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_info_for_orig_sample@726d84ed4e7280039be9c0e8ee30e9b0.pkl98da685df793d1bd97155fd1cc83e4a2.scramlbed
in ISIC OpISICSampleIDDecode _call_ sid= 
OpLoadImage, img file:  ./_examples/isic/data_dir\ISIC2019/ISIC_2019_Training_Input\ISIC_0072648.jpg jpg


caching:  90%|█████████ | 9/10 [00:30<00:03,  3.34s/it]

in save safe hdf5 file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@6eee2b91b0abe3870a5a6038b7b94514.hdf5a8fe5bc2f09b1eaa7c7581c05c7d4ce5.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@6eee2b91b0abe3870a5a6038b7b94514.pkl.gzadffc8ddb3f58aff43a8f9c2514e41f2.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_info_for_orig_sample@6eee2b91b0abe3870a5a6038b7b94514.pklfc6322379c3f20adbe388878ba091d80.scramlbed
in ISIC OpISICSampleIDDecode _call_ sid= 
OpLoadImage, img file:  ./_examples/isic/data_dir\ISIC2019/ISIC_2019_Training_Input\ISIC_0072649.jpg jpg


caching: 100%|██████████| 10/10 [00:33<00:00,  3.37s/it]

in save safe hdf5 file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@9eb861eb3cc48fe29c0a849ea9665c0d.hdf520b41d6c190a374258ec512bb1e9b19a.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_sample_id@9eb861eb3cc48fe29c0a849ea9665c0d.pkl.gz5dfc19a75e764c9919855a1d5af33742.scramlbed
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\out_info_for_orig_sample@9eb861eb3cc48fe29c0a849ea9665c0d.pkl2f9a4545c2d2e1e8c7726482fcf8fec8.scramlbed
======== wrote ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\pipeline_hash_ba80db34b3fed6fa5a43284ddb7769b2_desc.txt
in save pickle file:  ./_examples/isic/cache_dir\isic_cache_ver0\hash_ba80db34b3fed6fa5a43284ddb7769b2\full_sets_info\samples_ids_hash@437650e084bf7640c0f0882589810bf9.pkl.gze335322e163d087a96db613655fca950.scramlbed


In [ ]:
    # infer
    if "infer" in RUNNING_MODES:
        run_infer(paths=PATHS, infer_common_params=INFER_COMMON_PARAMS)

    # eval
    if "eval" in RUNNING_MODES:
        run_eval(paths=PATHS, eval_common_params=EVAL_COMMON_PARAMS)

In [ ]:
torch.cuda.is_available()